In [2]:
import numpy as np
import pandas as pd
from shekar import Normalizer, WordTokenizer
from shekar.preprocessing import  EmojiRemover, PunctuationRemover
from tqdm import tqdm
tqdm.pandas()
import time

In [3]:
products_df = pd.read_csv("/mnt/e/genAI/4supervised_ML_workshop/samsung_products.csv")

comments_df = pd.read_csv("/mnt/e/genAI/4supervised_ML_workshop/samsung_comments.csv")

print(products_df.shape, comments_df.shape)

(900, 13) (13723, 7)


In [4]:
# Merge comments with product details using product_id as the key (left join to keep all comments)
merged = pd.merge(comments_df, products_df, left_on='product_id', right_on='id', how='left')
print(merged.shape)
merged.head()

(13723, 20)


,product_id,comment_id,title,body,rate,date,user_name,id,title_fa,title_en,brand,category,status,selling_price,rrp_price,rating,rating_count,recommendation_percentage,color,url
0,16736269,80063210,NaN,گوشی مطابق رنگ انتخابی ارسال . به درستی و آسان...,5.0,22 آبان 1404,حمیدرضا زوری پور,16736269,گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,NaN,NaN,marketable,555000000.0,555000000.0,90.47,3436,NaN,خاکستری,/product/dkp-16736269/گوشی-موبایل-سامسونگ-مدل-...
1,16736269,80050936,NaN,ممنون از دیجی. از خریدم راضی بودم.,3.0,22 آبان 1404,میلاد عسکری,16736269,گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,NaN,NaN,marketable,555000000.0,555000000.0,90.47,3436,NaN,خاکستری,/product/dkp-16736269/گوشی-موبایل-سامسونگ-مدل-...
2,16736269,80031360,NaN,چقد طول میکشه تا فول شارژ بشه,0.0,21 آبان 1404,کاربر دیجی‌کالا,16736269,گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,NaN,NaN,marketable,555000000.0,555000000.0,90.47,3436,NaN,خاکستری,/product/dkp-16736269/گوشی-موبایل-سامسونگ-مدل-...
3,16736269,80025754,NaN,گوشی خوبیه کیفیت صفحه بنظر من که گوشی قبلم a52...,5.0,21 آبان 1404,رضا فرهادی,16736269,گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,NaN,NaN,marketable,555000000.0,555000000.0,90.47,3436,NaN,خاکستری,/product/dkp-16736269/گوشی-موبایل-سامسونگ-مدل-...
4,16736269,80012297,NaN,گوشیه خوبی ولی کاش قیمتش کمی مناسب تر بود و ای...,5.0,21 آبان 1404,رضا جوادزاده,16736269,گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,NaN,NaN,marketable,555000000.0,555000000.0,90.47,3436,NaN,خاکستری,/product/dkp-16736269/گوشی-موبایل-سامسونگ-مدل-...


In [5]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13723 entries, 0 to 13722
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 13723 non-null  int64  
 1   comment_id                 13723 non-null  int64  
 2   title                      4593 non-null   object 
 3   body                       13723 non-null  object 
 4   rate                       13723 non-null  float64
 5   date                       13723 non-null  object 
 6   user_name                  13723 non-null  object 
 7   id                         13723 non-null  int64  
 8   title_fa                   13723 non-null  object 
 9   title_en                   13658 non-null  object 
 10  brand                      0 non-null      float64
 11  category                   0 non-null      float64
 12  status                     13723 non-null  object 
 13  selling_price              1799 non-null   flo

In [6]:
# Drop unnecessary columns to simplify the dataset
merged.drop(columns=["comment_id", "title", "date", "user_name",
                      "id", "title_fa", "brand", "category",
                      "status", "recommendation_percentage", "url", "color"], inplace=True)


In [7]:
merged.head()

,product_id,body,rate,title_en,selling_price,rrp_price,rating,rating_count
0,16736269,گوشی مطابق رنگ انتخابی ارسال . به درستی و آسان...,5.0,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,555000000.0,555000000.0,90.47,3436
1,16736269,ممنون از دیجی. از خریدم راضی بودم.,3.0,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,555000000.0,555000000.0,90.47,3436
2,16736269,چقد طول میکشه تا فول شارژ بشه,0.0,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,555000000.0,555000000.0,90.47,3436
3,16736269,گوشی خوبیه کیفیت صفحه بنظر من که گوشی قبلم a52...,5.0,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,555000000.0,555000000.0,90.47,3436
4,16736269,گوشیه خوبی ولی کاش قیمتش کمی مناسب تر بود و ای...,5.0,Samsung Galaxy S24 FE Dual SIM Storage 256GB A...,555000000.0,555000000.0,90.47,3436


In [8]:
duplicate_rows = merged.duplicated()
print(duplicate_rows)

0        False
1        False
2        False
3        False
4        False
         ...  
13718    False
13719    False
13720    False
13721    False
13722    False
Length: 13723, dtype: bool


In [9]:
merged.isnull().sum()

product_id           0
body                 0
rate                 0
title_en            65
selling_price    11924
rrp_price        11924
rating               0
rating_count         0
dtype: int64

In [10]:
# Drop rows with missing values in key columns to clean the dataset
merged.dropna(subset=["title_en"], inplace=True)
    
# Drop additional unnecessary price columns as they may not be needed for sentiment analysis
merged.drop(columns=["selling_price", "rrp_price"], inplace=True)

In [11]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13658 entries, 0 to 13722
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    13658 non-null  int64  
 1   body          13658 non-null  object 
 2   rate          13658 non-null  float64
 3   title_en      13658 non-null  object 
 4   rating        13658 non-null  float64
 5   rating_count  13658 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 746.9+ KB


In [12]:
# Function to assign sentiment labels based on rating
def label_from_rate(rate):
    if rate <= 2:
        return "negative"
    elif rate == 3:
        return "neutral"
    else:
        return "positive"

merged["sentiment"] = merged["rate"].apply(label_from_rate)

In [14]:
# Making a clean dataset
merged.to_csv("samsung_mobiles.csv", index=False, encoding="utf-8-sig")